In [2]:
import pandas as pd
import numpy as np

lots = pd.read_csv('../clean-files/artsy_auctions_artworks_info.csv')
lots.columns

Index(['url', 'img_url', 'Price', 'Artist', 'Artist_url', 'Title',
       'Pre-sale estimate', 'Medium', 'Sale Date', 'Auction house',
       'Price_USD', 'Height (cm)', 'Width (cm)', 'Area (cm²)', 'Price_fix',
       'Price_unit', 'Sale details', 'Price (USD)', 'Price (USD / cm²)',
       'Price (USD / cm)', 'Technique', 'Sold', 'Year', 'Year of sale'],
      dtype='object')

In [6]:
prints = lots[lots['Technique'] == 'print']
paintings = lots[lots['Technique'] == 'painting']
drawings = lots[lots['Technique'] == 'drawing']

print('Prints: ', len(prints))
print('Paintings: ', len(paintings))
print('Drawings: ', len(drawings))

Prints:  3898
Paintings:  1282
Drawings:  153


In [25]:
# Median price of each medium
prints_median_price = prints['Price (USD)'].median()
paintings_median_price = paintings['Price (USD)'].median()
drawings_median_price = drawings['Price (USD)'].median()
# Weight of each medium
prints_weight = prints_median_price / (prints_median_price + paintings_median_price + drawings_median_price)
paintings_weight = paintings_median_price / (prints_median_price + paintings_median_price + drawings_median_price)
drawings_weight = drawings_median_price / (prints_median_price + paintings_median_price + drawings_median_price)

print('Prints weight: ', prints_weight.round(2))
print('Paintings weight: ', paintings_weight.round(2))
print('Drawings weight: ', drawings_weight.round(2))

Prints weight:  0.03
Paintings weight:  0.91
Drawings weight:  0.06


In [48]:
artists_names = ['Marc Chagall', 'Victor Vasarely']

artists_indices = pd.DataFrame()

for artist_name in artists_names:
    print(artist_name)
    artist_lots = lots[lots['Artist'] == artist_name]

    # artist dataframe: index: years (from first to last lot), columns: prints_median_price, paintings_median_price, drawings_median_price
    artist_df = pd.DataFrame(columns=['prints_median_price', 'paintings_median_price', 'drawings_median_price'])
    artist_df['prints_median_price'] = prints[prints['Artist'] == artist_name].groupby('Year of sale')['Price (USD)'].median()
    artist_df['paintings_median_price'] = paintings[paintings['Artist'] == artist_name].groupby('Year of sale')['Price (USD)'].median()
    artist_df['drawings_median_price'] = drawings[drawings['Artist'] == artist_name].groupby('Year of sale')['Price (USD)'].median()

    artist_df = artist_df.fillna(0)

    artist_df['prints_weight'] = np.where(artist_df['prints_median_price'] == 0, 0, prints_weight)
    artist_df['paintings_weight'] = np.where(artist_df['paintings_median_price'] == 0, 0, paintings_weight)
    artist_df['drawings_weight'] = np.where(artist_df['drawings_median_price'] == 0, 0, drawings_weight)

    artist_df['total_weight'] = artist_df['prints_weight'] + artist_df['paintings_weight'] + artist_df['drawings_weight']

    artist_df['Price Index'] = (
        artist_df['prints_weight'] * artist_df['prints_median_price'] +
        artist_df['paintings_weight'] * artist_df['paintings_median_price'] +
        artist_df['drawings_weight'] * artist_df['drawings_median_price']
    ) / artist_df['total_weight']

    # apply 3-year moving average
    artist_df['Price Index'] = artist_df['Price Index'].rolling(window=3).mean()

    # Assign the relevant portion of 'Price Index' to the corresponding column in 'artists_indices'
    artists_indices[artist_name] = artist_df['Price Index']

artists_indices.round(2)

Marc Chagall
Victor Vasarely


,Marc Chagall,Victor Vasarely
Year of sale,,
1993,NaN,NaN
1995,NaN,NaN
1996,304265.12,NaN
1997,301004.77,NaN
1998,246579.26,NaN
1999,264444.64,NaN
2000,254779.86,NaN
2001,342668.65,NaN
2002,285967.59,NaN
